<a href="https://colab.research.google.com/github/ysthakur/fakenews/blob/main/fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
import pandas as pd
import nltk

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

# Loading data

In [ ]:
import re

# Split rows of train.csv into [text, label] if possible
def split_text_label(text):
  # The first group is the text, the second is the label at the end
  match = re.match(r'^(.*)\s+?([^\s]+)$', text)
  if not match: raise ValueError(f'format wrong arg={text}')
  text = match.group(1)
  label = match.group(2)
  if label == '1': label = 1.0
  elif label == '0': label = 0.0
  elif label == 'label': return None # There's a line "content label" in the data
  else: raise ValueError(f'label wrong label={label}, text={text}')
  return [label, text.strip()]

def load_csv(file_name):
  # data obtained from https://www.kaggle.com/c/fakenewskdd2020/data
  with open(f'/content/drive/MyDrive/fakenewskdd2020/{file_name}.csv', 'r') as csv:
    next(csv) # Skip the header with the column titles
    return pd.Series(line for line in csv)

train_raw = load_csv('train').apply(split_text_label)
train_raw.dropna(inplace=True)
# X_train = pd.DataFrame(row for row in train)
X_train = train_raw.apply(lambda text_label: text_label[1])
y_train = train_raw.apply(lambda text_label: text_label[0])
X_test = load_csv('test').str.replace(re.compile('^\\d+\t'), '')
y_test = load_csv('sample_submission').str.replace(re.compile('^\\d+,|\n'), '').apply(float)

assert X_train.shape() == y_train.shape()
assert X_test.shape() == y_test.shape()
X_train, y_train, X_test, y_test

(0       Get the latest from TODAY Sign up for our news...
 1       2d  Conan On The Funeral Trump Will Be Invited...
 2       It’s safe to say that Instagram Stories has fa...
 3       Much like a certain Amazon goddess with a lass...
 4       At a time when the perfect outfit is just one ...
                               ...                        
 4983    The storybook romance of WWE stars John Cena a...
 4984    The actor told friends he’s responsible for en...
 4985    Sarah Hyland is getting real.  The Modern Fami...
 4986    Production has been suspended on the sixth and...
 4987    A jury ruled against Bill Cosby in his sexual ...
 Length: 4987, dtype: object, 0       1.0
 1       1.0
 2       0.0
 3       0.0
 4       0.0
        ... 
 4983    0.0
 4984    0.0
 4985    0.0
 4986    0.0
 4987    0.0
 Length: 4987, dtype: float64, 0       The 2017 Teen Choice Awards ceremony was held ...
 1       The concert, part of “The Joshua Tree Tour,” w...
 2       Selena Gomez refuses t

# Processing text

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')

